## SEI growth under cycling conditions

SEI growth is different under storage and cycling. In this case we are going to simulate the cycling of the cell with the solvent-diffusion SEI model active.

In [8]:
from cideMOD import CSI, CellParser, get_model_options

model_options = get_model_options(model='P2D', SEI_model='solvent_diffusion',
                                  save_path="results/06_cycling_SEI")

Saving results to /home/fenicsx/shared/cideMOD/examples/notebooks/results/results_06_cycling_SEI


In this case we are going to create a cycling test plan using python dictionaries. Let's build the
test plan step by step.

1. Discharge at 0.5C until 2V or 2 h

In [9]:
discharge_trigger = {
    "type": "Voltage",
    "value": 2,
    "unit": "V",
    "atol": 1e-4, # Absolute tolerance
    "rtol": 1e-3, # Relative tolerance
    "goto": "Next" 
}

discharge_input = {
    "name": "Discharge",
    "type": "Current",
    "value": -0.5,
    "unit": "C",
    "t_max": {"value": 2, "unit": "h"},
    "store_delay": -1,
    "min_step": 10,
    "max_step": 60,
    "time_adaptive_tol": 1e-2,
    "adaptive": True,
    "events": [discharge_trigger]
}

2. A CC charge at 0.5C until 4.2 V or 2 h

In [10]:
charge_CC_trigger = {
    "type": "Voltage",
    "value": 4.2,
    "unit": "V",
    "atol": 1e-4, 
    "rtol": 1e-3,
    "goto": "Next" 
}

charge_CC_input = {
    "name": "Charge-CC",
    "type": "Current",
    "value": 0.5,
    "unit": "C",
    "t_max": {"value": 2, "unit": "h"},
    "store_delay": -1,
    "min_step": 10,
    "max_step": 60,
    "time_adaptive_tol": 1e-2,
    "adaptive": True,
    "events": [charge_CC_trigger]
}

3. A CV charge at 4.2 V until 0.05C or 2 h

In [11]:
charge_CV_trigger = {
    "type": "Current",
    "value": 0.05,
    "unit": "C",
    "atol": 1e-4, 
    "rtol": 1e-3,
    "goto": "Next" 
}

charge_CV_input = {
    "name": "Charge-CV",
    "type": "Voltage",
    "value": 4.2,
    "unit": "V",
    "t_max": {"value": 2, "unit": "h"},
    "store_delay": -1,
    "min_step": 10,
    "max_step": 60,
    "time_adaptive_tol": 1e-2,
    "adaptive": True,
    "events": [charge_CV_trigger]
}

Now we can build the test plan

In [12]:
N_cycles = 5
test_plan = {
    "initial_state": {"SOC": 1, "exterior_temperature": 298.15},
    "steps": [
        {
            "name": "Cycling",
            "type": "Cycle",
            "count": N_cycles,
            "steps": [
                discharge_input,
                charge_CC_input,
                charge_CV_input,
            ]
        }
    ]
}

Finally, we select the `Safari 2009` dataset, modifying it slightly to move on to the reaction-limited
regime and run the simulation. $^{[1]}$

In [13]:
data_path = "../../data/data_Safari_2009"
cell_data = "params_cycling_lumped.json"
cell = CellParser(cell_data, data_path, model_options)
cell.update_parameters({
    "cell.anode.SEI.porous.solvent_diffusion": 1.3e-15,
    "cell.anode.SEI.porous.rate_constant" : 1.36e-12
})


Negative electrode capacity: 1.697784
Positive electrode capacity: 2.163497
Cell capacity: 1.697784
Negative electrode capacity: 1.697784
Positive electrode capacity: 2.163497
Cell capacity: 1.697784


In [14]:
csi = CSI(cell, model_options, test_plan)
status = csi.run_test_plan()

Building mesh for P2D problem with 3 components and 90 nodes
Finished mesh construction
Initial state:
	SOC : 1
	exterior_temperature : 298.15
Steps:
0 -  Cycle 'Cycling' repeats 5 times:
	 0 - CC_Discharge: Apply -0.8488917862020671 A during 2 hours until [Voltage Trigger at 2 V]
	 1 - CC_Charge-CC: Apply 0.8488917862020671 A during 2 hours until [Voltage Trigger at 4.2 V]
	 2 - CV_Charge-CV: Apply 4.2 V during 2 hours until [Current Trigger at 0.085 A]
Building problem setup


- Building cell parameters - Done   
- Initializing state - Done         
- Build variational formulation - Done 
-- Cycle 'Cycling', iteration number 0 --
CC_Discharge: Apply -0.8488917862020671 A during 2 hours until [Voltage Trigger at 2 V]
Reached max time 7200.00                            

CC_Charge-CC: Apply 0.8488917862020671 A during 2 hours until [Voltage Trigger at 4.2 V]
Voltage is 4.2 V at 3h 35min 11s                    

CV_Charge-CV: Apply 4.2 V during 2 hours until [Current Trigger at 0.085 A]
Current is 0.085 A at 4h 52min 24s                  

-- Cycle 'Cycling', iteration number 1 --           
CC_Discharge: Apply -0.8488917862020671 A during 2 hours until [Voltage Trigger at 2 V]
Reached max time 24748.77                           

CC_Charge-CC: Apply 0.8488917862020671 A during 2 hours until [Voltage Trigger at 4.2 V]
Voltage is 4.2 V at 8h 25min 9s                     

CV_Charge-CV: Apply 4.2 V during 2 hours until [Current Trigger at 0.085 A]
Current is 0.08

In [15]:
csi.plot_global_results()

[1] M. Safari et al 2009 J. Electrochem. Soc. 156 A145 doi: 10.1149/1.3043429